<a href="https://colab.research.google.com/github/nghitct/AlgorithmSupportedInductionPersonality/blob/main/Explainability_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import basic libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')
from itertools import combinations, product
from google.colab import drive 
drive.mount('/content/gdrive')
import networkx as nx 
import random

Mounted at /content/gdrive


In [ ]:
#function for generating graph: 
# gtype: type of graph, can be 'random' (Erdos-Renyi model with n nodes and prob p),
# 'smallworld' (Watts Strogatz model with n nodes, k mean degrees and beta),
# 'scalefree' (power law graph by Barabasi Albert model with n nodes and m 
# number of edges to attach from a new node to existing node), 
# 'clusteredscalefree' (Holme Kim model for power law graph with approximate average clustering)

def _gen_graph(gtype,gparams):
  # Erdos Renyi random graph
  if gtype=='random':
    G=nx.erdos_renyi_graph(gparams['n'],gparams['p'])
  # Watts Strogatz small world graph
  elif gtype=='smallworld':
    G=nx.watts_strogatz_graph(gparams['n'],gparams['k'],gparams['beta'])
  # Barabasi Albert scale free graph
  elif gtype=='scalefree':
    G=nx.barabasi_albert_graph(gparams['n'],gparams['m'])
  # clustered scale free graph
  elif gtype=='clusteredscalefree':
    G=nx.powerlaw_cluster_graph(gparams['n'],gparams['m'],gparams['p'])
  else:
    G=nx.empty_graph(gparams['n'])
    print('type error')
  # remove loop and parallel edges
  G.remove_edges_from(nx.selfloop_edges(G))
  G=nx.Graph(G)
  return G

#function for selecting explanatory nodes K (overlapping nodes):
# spec includes 'type' and 'measure'; G: explainer's graph; k: size of K
# nodes can be selected randomly ('type'='random') or proportionally to some
# network measures ('type'='central') or inversely proportionally ('type'='invcentral').
# 4 types of measures: 'degree', 'closeness', 'pagerank', 'betweenness'

def _sel_exp_nodes(spec,G,k,seed):
  gen = np.random.RandomState(seed)
  # pick node randomly
  if spec['type']=='random':
    K=gen.choice(list(G.nodes),k,replace=False)
  # pick node proportionally to central measures
  elif spec['type']=='central':
    # degree measure
    if spec['measure']=='degree':
      c=[x[1] for x in G.degree()]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    # pagerank centrality
    elif spec['measure']=='pagerank':
      c=[x for x in nx.pagerank_numpy(G,alpha=0.75)]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    # closeness centrality
    elif spec['measure']=='closeness':
      c=[x for x in nx.closeness_centrality(G).values()]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    # betweenness centrality
    elif spec['measure']=='betweenness':
      c=[x for x in nx.betweenness_centrality(G).values()]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    else:
      K=list()
      print('measure error')
  # pick node inversely proportionally to central measures
  elif spec['type']=='invcentral':
    # degree measure
    if spec['measure']=='degree':
      c=[1/x[1] if x[1]!=0 else 0 for x in G.degree()]    
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    # pagerank centrality
    elif spec['measure']=='pagerank':
      c=[1/x if x!=0 else 0 for x in nx.pagerank_numpy(G,alpha=0.75)]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    # closeness centrality
    elif spec['measure']=='closeness':
      c=[1/x if x!=0 else 0 for x in nx.closeness_centrality(G).values()]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    # betweenness centrality
    elif spec['measure']=='betweenness':
      c=[1/x if x!=0 else 0 for x in nx.betweenness_centrality(G).values()]
      p=[x/sum(c) for x in c]
      K=gen.choice(list(G.nodes),k,replace=False,p=p)
    else:
      K=list()
      print('measure error')
  else:
    K=list()
    print('type error')
  return K

#function to generate search list: list of nodes to search for one node 
#to reveal to explainee at each trial
# G: explainer's graph; target node: node to be explained;
# level: level of ego-network of target node for searching;
# expl_path: nodes already revealed to explainee

def _gen_search_list (G,target_node,level,expl_path):
  # get all nodes in the l-level ego graph of the target node
  full_search_list=list(nx.ego_graph(G,target_node,radius=level).nodes)
  # remove nodes that are already revealed to explainee (nodes in explaining path)
  search_list=list(set(full_search_list).symmetric_difference(set(expl_path)))
  return search_list

#function to select one node in the searh list
# search_list: list of nodes to search for one node to reveal to explainee;
# G: explainer's graph; target node: node to be explained;
# behavior: 0: choose randomly, 1: choose proportionally to the number of shortest
# paths between the target node and nodes in the search list 
def _choose_trial(search_list,G,target_node,behavior,seed):
  gen = np.random.RandomState(seed)
  # choose randomly (and uniformly)
  if behavior == 0:
    trial = gen.choice(search_list)
  # nodes with more shortest paths to the target node 
  # will have higher probability of being selected
  elif behavior == 1:
    # get numbers of shortest paths from the target node to nodes in the search list
    p=[len(list(nx.all_shortest_paths(G,R0,x))) for x in search_list]
    # turn to probablity
    p=[x/sum(p) for x in p]
    # choose nodes with probability p
    trial = gen.choice(search_list,p=p)
  return trial

#function for search process
# G: explainer's graph, target_node: node to be explained; behavior: 0 or 1
# rule: 
def _search_process (G,target_node,behavior,seed):
  # initial values
  not_explain=True
  not_exhaus=True
  expl_path=list()
  level=0

  # find the highest radius of the ego network of the target node
  max_radius=max(nx.single_source_shortest_path_length(G,target_node).values())

  # when explainability has not been achieved and not all nodes in the (largest)
  # target node's ego graph are revealed:
  while not_explain and not_exhaus:
    # generate search list
    search_list=_gen_search_list(G,target_node,level,expl_path)
    # pick one node in the search list
    trial=_choose_trial(search_list,G,target_node,behavior,seed)
    # if trial is in the explanatory set K: 
    # explainability is achieved -> not_explain = True
    if trial in K:
      expl_path.append(trial)
      not_explain=False
    # if trial is not in the explanatory set K:
    else:
      # add trial to the explaining path expl_path:
      expl_path.append(trial)
      # remove trial out of the search list:
      search_list.remove(trial)
      # if the search list is empty (all nodes in the search list are revealed)
      if len(search_list)==0:
        # if we still have othe levels to search (level<max_radius): move one level up
        if level<max_radius:
          level=level+1
        # if all levels are used: not_exhaus = False
        else:
          not_exhaus=False

  # gen and return results 
  results={'explain': (not not_explain),
         'time': len(expl_path),
         'radius': level,
         'explain_path': expl_path}
  return results

def _ave_shortest_path (G):
  if nx.is_connected(G):
    return(nx.average_shortest_path_length(R))
  else:
    return(9999)

# Round 1: Random graph, random overlap, random search

## Set parameters

In [ ]:
#set seed and randomstate
seed_list = random.sample(range(1,1000000),1000)

#We have a domain S with size N_S, the explainer's graph with size N_R,
# the explainee's graph with size N_E and the probability p_K of overlapping nodes
# (from the explainer's graph R). If we let N_R and p_K run without any constraint,
# we have to ignore N_S and N_E. For domain S, we can assume unlimited domain 
# (or domain is very large compared with individual knowledge graph). For N_E, we
# can assume p_K is the joint probability of both overlapping and size of the 
# explainee's graph. So in the following codes, we ignore N_S and N_E. If later,
# we want to extend the model to incorporate the explainee's graph E or the domain 
# S (i.e. S is not fully connected as we assume now), we need to add more parameters.

#define the explainer's graph
N_R_list = list(np.arange(5,105))
p_random_list = list(np.arange(0.1,1,0.1))

#define the set of explanatory nodes K
p_K_list = list(np.arange(0.1,1,0.1))

## Simulation

In [ ]:
results_all=pd.DataFrame(columns=['explain', 'time', 'radius', 'explain_path', 'density', 'ave_sh_path',
       'ave_clustering', 'efficient', 'ave_deg', 'size_explainer_graph',
       'type_explainer_graph', 'seed', 'type_overlap', 'prop_of_overlap',
       'search_behavior'])
for N_R in N_R_list:
  for p_random in p_random_list:
    for seed in seed_list:
      random.seed(seed)
      #create the explainer's graph
      R_params={'gtype': 'random','gparams':{'n': N_R,'p': p_random}}
      R = _gen_graph(**R_params)
      #calculate the graph properties
      R_properties={'density': nx.density(R),
                    'ave_sh_path': _ave_shortest_path(R),
                    'ave_clustering': nx.average_clustering(R),
                    'efficient': nx.global_efficiency(R), 
                    'ave_deg': sum([x[1] for x in list(R.degree)])/N_R}
      
      for p_K in p_K_list:
        #define K explanatory nodes are chosen from the explainer's knowledge graph
        N_K = round(N_R*p_K)
        params={'spec': {'type': 'random'},'G': R,'k': N_K, 'seed': seed}
        K=_sel_exp_nodes(**params)

        #define target node:
        gen = np.random.RandomState(seed)
        R0=gen.choice(list(R.nodes()))

        #produce search results
        search_results=_search_process(R,R0,0,seed)

        #create data frame
        results=pd.concat([pd.DataFrame([search_results]),pd.DataFrame([R_properties])],axis=1)
        results['size_explainer_graph']=N_R
        results['type_explainer_graph']='random'
        results['seed']=seed
        results['type_overlap']='random'
        results['prop_of_overlap']=p_K
        results['search_behavior']=0
        
        #add to the full data frame
        results_all = pd.concat([results_all,results],ignore_index=True)

        path="/content/gdrive/MyDrive/Colab Notebooks/Expl-RanGr-RanOver-RanSearch-iteration-1228.csv"
        with open(path, 'w', encoding = 'utf-8-sig') as f:
          results_all.to_csv(f)

path="/content/gdrive/MyDrive/Colab Notebooks/Expl-RanGr-RanOver-RanSearch-1228.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  results_all.to_csv(f)

# Round 2: Random graph, random overlap, search proportional to number of shortest paths

# Initial parameters

In [ ]:
#set seed and randomstate
seed_list = random.sample(range(1,1000000),1000)

#We have a domain S with size N_S, the explainer's graph with size N_R,
# the explainee's graph with size N_E and the probability p_K of overlapping nodes
# (from the explainer's graph R). If we let N_R and p_K run without any constraint,
# we have to ignore N_S and N_E. For domain S, we can assume unlimited domain 
# (or domain is very large compared with individual knowledge graph). For N_E, we
# can assume p_K is the joint probability of both overlapping and size of the 
# explainee's graph. So in the following codes, we ignore N_S and N_E. If later,
# we want to extend the model to incorporate the explainee's graph E or the domain 
# S (i.e. S is not fully connected as we assume now), we need to add more parameters.

#define the explainer's graph
N_R_list = list(np.arange(5,105))
p_random_list = list(np.arange(0.1,1,0.1))

#define the set of explanatory nodes K
p_K_list = list(np.arange(0.1,1,0.1))

## Simulation

In [ ]:
results_all=pd.DataFrame(columns=['explain', 'time', 'radius', 'explain_path', 'density', 'ave_sh_path',
       'ave_clustering', 'efficient', 'ave_deg', 'size_explainer_graph',
       'type_explainer_graph', 'seed', 'type_overlap', 'prop_of_overlap',
       'search_behavior'])
for N_R in N_R_list:
  for p_random in p_random_list:
    for seed in seed_list:
      random.seed(seed)
      #create the explainer's graph
      R_params={'gtype': 'random','gparams':{'n': N_R,'p': p_random}}
      R = _gen_graph(**R_params)
      #calculate the graph properties
      R_properties={'density': nx.density(R),
                    'ave_sh_path': _ave_shortest_path(R),
                    'ave_clustering': nx.average_clustering(R),
                    'efficient': nx.global_efficiency(R), 
                    'ave_deg': sum([x[1] for x in list(R.degree)])/N_R}
      
      for p_K in p_K_list:
        #define K explanatory nodes are chosen from the explainer's knowledge graph
        N_K = round(N_R*p_K)
        params={'spec': {'type': 'random'},'G': R,'k': N_K, 'seed': seed}
        K=_sel_exp_nodes(**params)

        #define target node:
        gen = np.random.RandomState(seed)
        R0=gen.choice(list(R.nodes()))

        #produce search results
        search_results=_search_process(R,R0,0,seed)

        #create data frame
        results=pd.concat([pd.DataFrame([search_results]),pd.DataFrame([R_properties])],axis=1)
        results['size_explainer_graph']=N_R
        results['type_explainer_graph']='random'
        results['seed']=seed
        results['type_overlap']='random'
        results['prop_of_overlap']=p_K
        results['search_behavior']=1
        
        #add to the full data frame
        results_all = pd.concat([results_all,results],ignore_index=True)

path="/content/gdrive/MyDrive/Colab Notebooks/Expl-RanGr-RanOver-PropSearch-1224.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  results_all.to_csv(f)

# Round 3: Small World graph, random overlap, random search

## Initial parameters

In [ ]:
#set seed and randomstate
seed_list = random.sample(range(1,1000000),1000)

#We have a domain S with size N_S, the explainer's graph with size N_R,
# the explainee's graph with size N_E and the probability p_K of overlapping nodes
# (from the explainer's graph R). If we let N_R and p_K run without any constraint,
# we have to ignore N_S and N_E. For domain S, we can assume unlimited domain 
# (or domain is very large compared with individual knowledge graph). For N_E, we
# can assume p_K is the joint probability of both overlapping and size of the 
# explainee's graph. So in the following codes, we ignore N_S and N_E. If later,
# we want to extend the model to incorporate the explainee's graph E or the domain 
# S (i.e. S is not fully connected as we assume now), we need to add more parameters.

#define the explainer's graph
N_R_list = list(np.arange(5,105))
p_random_list = list(np.arange(0.1,1,0.1))

#define the set of explanatory nodes K
p_K_list = list(np.arange(0.1,1,0.1))

## Simulation